In [3]:
import numpy as np
from contract import Contract, Cashflow, Leg, Fork
from observable import Observation, Observable, Ticker
from datetime import datetime

# class Contract

Base class to represent a Contract

# class CashFlow

Actual payment of a market observable. 


In [4]:
Libor6M = Ticker("EUR.LIBOR.6M", "Reuters")
fixing_date = datetime(2024, 7, 10)
payment_date = datetime(2024, 7, 12)
cf = Cashflow(Observation(Libor6M, fixing_date) + 0.002, payment_date, "EUR", 1_000_000)
cf

(EUR.LIBOR.6M(2024-07-10) + 0.002)	1000000	EUR	2024-07-12

In [5]:
cf2 = Cashflow(np.max(Observation(Libor6M, fixing_date) - 0.01, 0.01), payment_date, "EUR", 1_000_000)
print(cf2)

max((EUR.LIBOR.6M(2024-07-10) - 0.01), 0.01))	1000000	EUR	2024-07-12


## Spread option cash flow

In [6]:
Libor1M = Ticker("EUR.LIBOR.1M", "Reuters")
spread_cf = Cashflow(np.max(Observation(Libor6M, fixing_date) - Observation(Libor1M, fixing_date), 0.0), payment_date, "EUR", 1_000_000)


## Schedule

We use datetime library, but can also use something like QuantLib

In [5]:
from schedule import schedule
from dateutil.relativedelta import relativedelta

start_date = datetime(2024, 7, 10)
tenor = relativedelta(years=10)

coupon_leg_schedule = schedule(start_date, tenor, frequency=relativedelta(years=1))
coupon_fixings = [end - relativedelta(days=2) for _, end in coupon_leg_schedule]
funding_leg_schedule = schedule(start_date, tenor, frequency=relativedelta(months=6))
funding_fixings = [start - relativedelta(days=2) for start, _ in funding_leg_schedule]

call_dates = coupon_fixings[1:]
coupon_fixings

[datetime.datetime(2025, 7, 8, 0, 0),
 datetime.datetime(2026, 7, 8, 0, 0),
 datetime.datetime(2027, 7, 8, 0, 0),
 datetime.datetime(2028, 7, 8, 0, 0),
 datetime.datetime(2029, 7, 8, 0, 0),
 datetime.datetime(2030, 7, 8, 0, 0),
 datetime.datetime(2031, 7, 8, 0, 0),
 datetime.datetime(2032, 7, 8, 0, 0),
 datetime.datetime(2033, 7, 8, 0, 0),
 datetime.datetime(2034, 7, 8, 0, 0)]

In [6]:
CMS20Y = Ticker("EUR.CMS.20Y", "Reuters")
CMS2Y = Ticker("EUR.CMS.2Y", "Reuters")
Libor6M = Ticker("EUR.LIBOR.6M", "Reuters")


In [16]:
funding_leg = Leg([Cashflow(Observation(Libor6M, fixing) + 0.002, payment_date, "EUR", 1_000_000) for fixing, (_, payment_date) in zip(funding_fixings, funding_leg_schedule)])

In [8]:
coupons = [np.max(Observation(CMS20Y, fixing) - 2 * Observation(CMS2Y, fixing), 0.0) for fixing in coupon_fixings]


In [17]:
cpn_leg = Leg([Cashflow(coupon, payment_date, "EUR", 1_000_000) for coupon, (_, payment_date) in zip(coupons, coupon_leg_schedule)])

In [19]:
swap_contract = [funding_leg, cpn_leg]
swap_contract

[(EUR.LIBOR.6M(2024-07-08) + 0.002)	1000000	EUR	2025-01-10
 (EUR.LIBOR.6M(2025-01-08) + 0.002)	1000000	EUR	2025-07-10
 (EUR.LIBOR.6M(2025-07-08) + 0.002)	1000000	EUR	2026-01-10
 (EUR.LIBOR.6M(2026-01-08) + 0.002)	1000000	EUR	2026-07-10
 (EUR.LIBOR.6M(2026-07-08) + 0.002)	1000000	EUR	2027-01-10
 (EUR.LIBOR.6M(2027-01-08) + 0.002)	1000000	EUR	2027-07-10
 (EUR.LIBOR.6M(2027-07-08) + 0.002)	1000000	EUR	2028-01-10
 (EUR.LIBOR.6M(2028-01-08) + 0.002)	1000000	EUR	2028-07-10
 (EUR.LIBOR.6M(2028-07-08) + 0.002)	1000000	EUR	2029-01-10
 (EUR.LIBOR.6M(2029-01-08) + 0.002)	1000000	EUR	2029-07-10
 (EUR.LIBOR.6M(2029-07-08) + 0.002)	1000000	EUR	2030-01-10
 (EUR.LIBOR.6M(2030-01-08) + 0.002)	1000000	EUR	2030-07-10
 (EUR.LIBOR.6M(2030-07-08) + 0.002)	1000000	EUR	2031-01-10
 (EUR.LIBOR.6M(2031-01-08) + 0.002)	1000000	EUR	2031-07-10
 (EUR.LIBOR.6M(2031-07-08) + 0.002)	1000000	EUR	2032-01-10
 (EUR.LIBOR.6M(2032-01-08) + 0.002)	1000000	EUR	2032-07-10
 (EUR.LIBOR.6M(2032-07-08) + 0.002)	1000000	EUR	2033-01-

## Snowball example

In [21]:
snowball_cfs = np.cumsum(coupons)
snowball_cfs[0:3]


array([max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)),
       (max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)) + max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))),
       ((max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)) + max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))) + max((EUR.CMS.20Y(2027-07-08) - (EUR.CMS.2Y(2027-07-08) * 2)), 0.0)))],
      dtype=object)

# Handling optionality

## class Fork

Represents a switch from one contract to another. It's defined using the condition, contract1 and contract2.

The condition is an observable, it can be an expression computed using market indices or it can be based on either conterparty decision.

It's expressed as real value. If value is positive, contract1 is in force. If value is negative, contract2 is in force.

